In [1]:
import pandas as pd
import re
from gensim import corpora,models,similarities
from gensim.utils import tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
import numpy as np
from scipy import spatial
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [3]:
dirname = 'data'
files = os.listdir(dirname)
temp = list(map(lambda name: os.path.join(dirname, name), files))

парсим файл
1 положительные
0 нейтральные
-1 отрицательные

In [4]:
gen = {}
for file in temp:
    with open(file, 'r') as f:
        lst = []
        for line in f:
            book = ''
            for i in range(len(line)):
                if line[i] == ',' and line[i + 1] != ' ':
                    lst.append(book)
                    book = ''
                else:
                    book += line[i]
        if 'Posi' in file:
            gen['pos'] = lst
        elif 'Neut' in file:
            gen['net'] = lst
        else:
            gen['neg'] = lst

In [5]:
poz = {'0text0': gen['pos'], '0status0': 1}
net = {'0text0': gen['net'], '0status0': 0}
neg = {'0text0': gen['neg'], '0status0': -1}

создаем датафреймы

In [6]:
df = pd.DataFrame(poz)
df1 = pd.DataFrame(net)
df2 = pd.DataFrame(neg)
df = pd.concat([df, df1, df2])
df['0text0'] = df['0text0'].str.lower()
df.head(3)
# df = pd.DataFrame(io)
# df['text'] = df['text'].str.lower()
# df.head(3)
# df = pd.read_csv('data/processedPositive.csv', lineterminator=',', names=['text'])

,0text0,0status0
0,"an inspiration in all aspects: fashion, fitnes...",1
1,apka apna awam ka channel frankline tv aam adm...,1
2,beautiful album from the greatest unsung guit...,1


слова исключения

In [7]:
sw = stopwords.words('english')

токенизирование

In [8]:
def tokenize_in_df(strin):
    try:
        sp = list(tokenize(strin,lowercase=True, deacc=True,))
        return {w for w in sp if not w in sw} 
    except:
        return ""

In [9]:
def tokenize_in_df_lst(strin):
    try:
        sp = list(tokenize(strin,lowercase=True, deacc=True,))
        return [w for w in sp if not w in sw] 
    except:
        return ""

корень слова

In [10]:
def stemm_in_df(strin):
    sno = nltk.stem.SnowballStemmer('english')
    n = set()
    for w in strin:
        n.add(sno.stem(w))
    return(n)

In [11]:
def stemm_in_df_lst(strin):
    sno = nltk.stem.SnowballStemmer('english')
    n = []
    for w in strin:
        n.append(sno.stem(w))
    return(n)

словарная форма

In [12]:
def lemm_in_df(strin):
    lemma = nltk.stem.wordnet.WordNetLemmatizer()
    n = set()
    for w in strin:
        n.add(lemma.lemmatize(w))
    return(n)

In [13]:
def lemm_in_df_lst(strin):
    lemma = nltk.stem.wordnet.WordNetLemmatizer()
    n = []
    for w in strin:
        n.append(lemma.lemmatize(w))
    return(n)

создаем сеты и списки столбцы с полученными токенами

In [14]:
df["just tokenization"] = df["0text0"].apply(tokenize_in_df)
df["stemming"] = df["just tokenization"].apply(stemm_in_df)
df["lemmatization"] = df["just tokenization"].apply(lemm_in_df)
df["tokenz_lst"] = df["0text0"].apply(tokenize_in_df_lst)
df["stemm_lst"] = df["tokenz_lst"].apply(stemm_in_df_lst)
df["lemm_lst"] = df["tokenz_lst"].apply(lemm_in_df_lst)
df = df[df['just tokenization'] != set()]

In [15]:
df.head(3)

,0text0,0status0,just tokenization,stemming,lemmatization,tokenz_lst,stemm_lst,lemm_lst
0,"an inspiration in all aspects: fashion, fitnes...",1,"{aspects, beauty, fitness, kisses, thefashioni...","{aspect, fit, beauti, inspir, person, kiss, fa...","{aspect, beauty, fitness, personality, kiss, f...","[inspiration, aspects, fashion, fitness, beaut...","[inspir, aspect, fashion, fit, beauti, person,...","[inspiration, aspect, fashion, fitness, beauty..."
1,apka apna awam ka channel frankline tv aam adm...,1,"{production, visit, apka, page, channel, likes...","{aam, visit, product, apka, page, channel, awa...","{production, visit, apka, page, channel, frank...","[apka, apna, awam, ka, channel, frankline, tv,...","[apka, apna, awam, ka, channel, franklin, tv, ...","[apka, apna, awam, ka, channel, frankline, tv,..."
2,beautiful album from the greatest unsung guit...,1,"{backstage, time, genius, album, unsung, guita...","{time, genius, album, greatest, unsung, guitar...","{backstage, time, greatest, album, unsung, gui...","[beautiful, album, greatest, unsung, guitar, g...","[beauti, album, greatest, unsung, guitar, geni...","[beautiful, album, greatest, unsung, guitar, g..."


создаем токены из столбцов для определения нахождения в тексте

In [16]:
tokenz = set()
for index, row in df.iterrows():
    ss = row['just tokenization']
    for w in ss:
        tokenz.add(w)

In [17]:
stemi = set()
for index, row in df.iterrows():
    ss = row['stemming']
    for w in ss:
        stemi.add(w)

In [18]:
lemmi = set()
for index, row in df.iterrows():
    ss = row['lemmatization']
    for w in ss:
        lemmi.add(w)

In [19]:
df_token = df[['0text0', '0status0']]
df_stemm = df[['0text0', '0status0']]
df_lemm = df[['0text0', '0status0']]
# df_token.rename(columns={df_token.columns[0]:'1_text_[)1=*'}, inplace=True)
# df_stemm.rename(columns={df_stemm.columns[0]:'1_text_[)1=*'}, inplace=True)
# df_lemm.rename(columns={df_lemm.columns[0]:'1_text_[)1=*'}, inplace=True)

ищем вхождения (word counts)

In [20]:
for w in tokenz:
    df_token[w] = df_token["0text0"].str.count(f"[^a-z]{w}[^a-z]")

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [25]:
for w in stemi:
    df_stemm[w] = df_stemm["0text0"].str.count(f"{w}.*")

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [26]:
for w in lemmi:
    df_lemm[w] = df_lemm["0text0"].str.count(f"{w}.*")

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [27]:
df_lemm.head(3)

,0text0,0status0,nail,keep_smiling_y,admission,sp,restaurant,quoting,club,bsf,...,worry,ante,nuclear,dual,tweet,lacked,attractive,advertisement,rude,korean
0,"an inspiration in all aspects: fashion, fitnes...",1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,apka apna awam ka channel frankline tv aam adm...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,beautiful album from the greatest unsung guit...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ищем вхождения (1 or 0)

In [28]:
df_token_binar = df_token.copy(deep=True)
df_stemm_binar = df_stemm.copy(deep=True)
df_lemm_binar = df_lemm.copy(deep=True)

In [29]:
columns = list(df_token_binar)
for i in columns:
    if i != "0text0":
        df_token_binar[i] = np.where(df_token_binar[i] > 0, 1, 0)

In [30]:
columns = list(df_stemm_binar)
for i in columns:
    if i != "0text0":
        df_stemm_binar[i] = np.where(df_stemm_binar[i] > 0, 1, 0)

In [31]:
columns = list(df_lemm_binar)
for i in columns:
    if i != "0text0":
        df_lemm_binar[i] = np.where(df_lemm_binar[i] > 0, 1, 0)

Создадим словарь слов, которые есть во всем нашем наборе текстов:

In [100]:
dictionary = corpora.Dictionary(df["tokenz_lst"])
feature_cnt = len(dictionary.token2id)
dictionary.token2id

{'aspects': 0,
 'beauty': 1,
 'fashion': 2,
 'fitness': 3,
 'inspiration': 4,
 'kisses': 5,
 'personality': 6,
 'thefashionicon': 7,
 'aam': 8,
 'admi': 9,
 'apka': 10,
 'apna': 11,
 'awam': 12,
 'channel': 13,
 'fb': 14,
 'frankline': 15,
 'ka': 16,
 'likes': 17,
 'page': 18,
 'please': 19,
 'production': 20,
 'share': 21,
 'tv': 22,
 'visit': 23,
 'album': 24,
 'backstage': 25,
 'beautiful': 26,
 'genius': 27,
 'great': 28,
 'greatest': 29,
 'guitar': 30,
 'met': 31,
 'time': 32,
 'unsung': 33,
 'donate': 34,
 'good': 35,
 'luck': 36,
 'project': 37,
 'rich': 38,
 'riding': 39,
 'sunday': 40,
 'crying': 41,
 'joy': 42,
 'kissed': 43,
 'omg': 44,
 'anniv': 45,
 'happy': 46,
 'love': 47,
 'ming': 48,
 'papi': 49,
 'thanks': 50,
 'c': 51,
 'join': 52,
 'mon': 53,
 'singer': 54,
 'spread': 55,
 'tweeps': 56,
 'vote': 57,
 'word': 58,
 'review': 59,
 'smile': 60,
 'another': 61,
 'art': 62,
 'everything': 63,
 'know': 64,
 'need': 65,
 'picture': 66,
 'raffle': 67,
 'yay': 68,
 'amazing':

In [101]:
dictionary1 = corpora.Dictionary(df["stemm_lst"])
feature_cnt1 = len(dictionary1.token2id)
dictionary1.token2id

{'aspect': 0,
 'beauti': 1,
 'fashion': 2,
 'fit': 3,
 'inspir': 4,
 'kiss': 5,
 'person': 6,
 'thefashionicon': 7,
 'aam': 8,
 'admi': 9,
 'apka': 10,
 'apna': 11,
 'awam': 12,
 'channel': 13,
 'fb': 14,
 'franklin': 15,
 'ka': 16,
 'like': 17,
 'page': 18,
 'pleas': 19,
 'product': 20,
 'share': 21,
 'tv': 22,
 'visit': 23,
 'album': 24,
 'backstag': 25,
 'genius': 26,
 'great': 27,
 'greatest': 28,
 'guitar': 29,
 'met': 30,
 'time': 31,
 'unsung': 32,
 'donat': 33,
 'good': 34,
 'luck': 35,
 'project': 36,
 'rich': 37,
 'ride': 38,
 'sunday': 39,
 'cri': 40,
 'joy': 41,
 'omg': 42,
 'anniv': 43,
 'happi': 44,
 'love': 45,
 'ming': 46,
 'papi': 47,
 'thank': 48,
 'c': 49,
 'join': 50,
 'mon': 51,
 'singer': 52,
 'spread': 53,
 'tweep': 54,
 'vote': 55,
 'word': 56,
 'review': 57,
 'smile': 58,
 'anoth': 59,
 'art': 60,
 'everyth': 61,
 'know': 62,
 'need': 63,
 'pictur': 64,
 'raffl': 65,
 'yay': 66,
 'amaz': 67,
 'check': 68,
 'citi': 69,
 'egypt': 70,
 'hello': 71,
 'hope': 72,
 '

In [102]:
dictionary2 = corpora.Dictionary(df["lemm_lst"])
feature_cnt2 = len(dictionary2.token2id)
dictionary2.token2id

{'aspect': 0,
 'beauty': 1,
 'fashion': 2,
 'fitness': 3,
 'inspiration': 4,
 'kiss': 5,
 'personality': 6,
 'thefashionicon': 7,
 'aam': 8,
 'admi': 9,
 'apka': 10,
 'apna': 11,
 'awam': 12,
 'channel': 13,
 'fb': 14,
 'frankline': 15,
 'ka': 16,
 'like': 17,
 'page': 18,
 'please': 19,
 'production': 20,
 'share': 21,
 'tv': 22,
 'visit': 23,
 'album': 24,
 'backstage': 25,
 'beautiful': 26,
 'genius': 27,
 'great': 28,
 'greatest': 29,
 'guitar': 30,
 'met': 31,
 'time': 32,
 'unsung': 33,
 'donate': 34,
 'good': 35,
 'luck': 36,
 'project': 37,
 'rich': 38,
 'riding': 39,
 'sunday': 40,
 'cry': 41,
 'joy': 42,
 'kissed': 43,
 'omg': 44,
 'anniv': 45,
 'happy': 46,
 'love': 47,
 'ming': 48,
 'papi': 49,
 'thanks': 50,
 'c': 51,
 'join': 52,
 'mon': 53,
 'singer': 54,
 'spread': 55,
 'tweeps': 56,
 'vote': 57,
 'word': 58,
 'review': 59,
 'smile': 60,
 'another': 61,
 'art': 62,
 'everything': 63,
 'know': 64,
 'need': 65,
 'picture': 66,
 'raffle': 67,
 'yay': 68,
 'amazing': 69,
 '

Теперь нужно создать корпус, превратив наши токенизированные тексты в векторы

In [103]:
corpus = [dictionary.doc2bow(text) for text in df["tokenz_lst"]]
corpus1 = [dictionary1.doc2bow(text) for text in df["stemm_lst"]]
corpus2 = [dictionary2.doc2bow(text) for text in df["lemm_lst"]]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1)],
 [(24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(28, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)],
 [(41, 1), (42, 1), (43, 1), (44, 1)],
 [(45, 1), (46, 2), (47, 2), (48, 1), (49, 1)],
 [(46, 1), (50, 1)],
 [(51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)],
 [(28, 1), (50, 1), (59, 1), (60, 1)],
 [(61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)],
 [(23, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1)],
 [(77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1)],
 [(46, 1), (86, 1), (87, 1), (88, 1), (89, 1)],
 [(32, 1), (46, 1), (90, 1)

Именно этим займется модель tf-idf. Сама по себе аббревиатура TF-IDF расшифровывается как TF — term frequency, IDF — inverse document frequency, то есть отношение частоты употребления слова в отдельном тексте к частоте употребления слова во всех документах. Построенная на основе такой меры модель прекрасно подходит для поиска похожих текстов, поскольку позволяет сравнивать совокупные меры текстов между собой, строя матрицу похожести

In [104]:
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features = feature_cnt)
tfidf1 = models.TfidfModel(corpus1)
index1 = similarities.SparseMatrixSimilarity(tfidf1[corpus1],num_features = feature_cnt1)
tfidf2 = models.TfidfModel(corpus2)
index2 = similarities.SparseMatrixSimilarity(tfidf2[corpus2],num_features = feature_cnt2)

Как именно отработала эта модель мы сможем увидеть на примере, построив векторы наших опорных текстов и получив значения их похожестей из матрицы

In [105]:
df_tf_tokenz = df[['0text0', '0status0']]
df_tf_stemm = df[['0text0', '0status0']]
df_tf_lemm = df[['0text0', '0status0']]

In [106]:
for text in df['0text0']:
    kw_vector = dictionary.doc2bow(tokenize(text))
    df_tf_tokenz[text] = index[tfidf[kw_vector]]
    kw_vector = dictionary1.doc2bow(tokenize(text))
    df_tf_stemm[text] = index1[tfidf1[kw_vector]]
    kw_vector = dictionary2.doc2bow(tokenize(text))
    df_tf_lemm[text] = index2[tfidf2[kw_vector]]

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [114]:
df_tf_tokenz.head(3)

,0text0,0status0,"an inspiration in all aspects: fashion, fitness, beauty and personality. :)kisses thefashionicon",apka apna awam ka channel frankline tv aam admi production please visit or likes share :)fb page :...,beautiful album from the greatest unsung guitar genius of our time - and i've met the great backstage,good luck to rich riding for great project in this sunday. can you donate?,omg he... kissed... him crying with joy,happy anniv ming and papi!!!!! love love happy,thanks happy,"c'mon tweeps, join vote for the singer! do spread the word. :d",...,"oh jeez. i didn't even try taking a crack at it because i expected the build to be so bad, ha unhappy",r.i.p victoria wood heaven knows we're miserable now unhappy,that was a fun night! sorry i missed last night unhappy we really need to sync our calendars with and,when will you notice me? unhappy,"same unhappy , whats crazy is when i walked through them doors every administration + her was there that night. so why are her a",and yet if parents invest in child's emotional education by taking child out of school on holiday early that's un,i want more orientation unhappy,unhappy they not,wish knock out lang talaga for the new school year are good and cooperative groupmates please unhappy,i miss so much unhappy
0,"an inspiration in all aspects: fashion, fitnes...",1,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,apka apna awam ka channel frankline tv aam adm...,1,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026039,0.0
2,beautiful album from the greatest unsung guit...,1,0.0,0.0,1.0,0.026663,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


определяем топ 10 схожих по каждому признаку

In [111]:
res = {}
ls_res = []
for index, row in df.iterrows():
    first = row["just tokenization"]
    for index1, row1 in df.iterrows():
        second = row1["just tokenization"]
        l1 =[];l2 =[]
        if second == first:
            continue
        # сформировать набор, содержащий ключевые слова обеих строк
        rvector = first.union(second) 

        for w in rvector:
            if w in first: l1.append(1) # создать вектор

            else: l1.append(0)

            if w in second: l2.append(1)

            else: l2.append(0)
        # формула косинуса
        result = 1 - spatial.distance.cosine(l1, l2)
        res = {'first': row['0text0'], 'second': row1['0text0'], 'result': result}
        ls_res.append(res)
df_cos = pd.DataFrame(ls_res)
df_cos.sort_values(by=['result'], ascending=False, inplace=True)
df_cos.head(10)

,first,second,result
8096467,hi! we tried to call your number but got no re...,hi ashish! we tried to call your number but go...,0.966092
8339189,hi! we tried to call your number but got no re...,hi ashish! we tried to call your number but go...,0.966092
8803646,hi! we tried to call your number but got no re...,hi ashish! we tried to call your number but go...,0.966092
8804046,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8096867,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8339589,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
7530469,hi ashish! we tried to call your number but go...,hi! we tried to call your number but got no re...,0.966092
7530388,hi ashish! we tried to call your number but go...,hi! we tried to call your number but got no re...,0.966092
8734904,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8734985,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092


In [115]:
res = {}
ls_res = []
for index, row in df.iterrows():
    first = row["stemming"]
    for index1, row1 in df.iterrows():
        second = row1["stemming"]
        l1 =[];l2 =[]
        if second == first:
            continue
        # сформировать набор, содержащий ключевые слова обеих строк
        rvector = first.union(second) 

        for w in rvector:
            if w in first: l1.append(1) # создать вектор

            else: l1.append(0)

            if w in second: l2.append(1)

            else: l2.append(0)
        # формула косинуса
        result = 1 - spatial.distance.cosine(l1, l2)
        res = {'first': row['0text0'], 'second': row1['0text0'], 'result': result}
        ls_res.append(res)
df_cos1 = pd.DataFrame(ls_res)
df_cos1.sort_values(by=['result'], ascending=False, inplace=True)
df_cos1.head(10)

,first,second,result
8096792,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8734828,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8096392,hi! we tried to call your number but got no re...,hi ashish! we tried to call your number but go...,0.966092
7530315,hi ashish! we tried to call your number but go...,hi! we tried to call your number but got no re...,0.966092
7530396,hi ashish! we tried to call your number but go...,hi! we tried to call your number but got no re...,0.966092
7530551,hi ashish! we tried to call your number but go...,hi! we tried to call your number but got no re...,0.966092
8339514,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8735063,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8734909,hi! we tried to call your number but got no re...,hi! we tried to call your number but got no re...,0.966092
8803570,hi! we tried to call your number but got no re...,hi ashish! we tried to call your number but go...,0.966092


In [186]:
res = {}
ls_res = []
for index, row in df.iterrows():
    first = row["lemmatization"]
    for index1, row1 in df.iterrows():
        second = row1["lemmatization"]
        l1 =[];l2 =[]
        if second == first:
            continue
        # сформировать набор, содержащий ключевые слова обеих строк
        rvector = first.union(second) 

        for w in rvector:
            if w in first: l1.append(1) # создать вектор

            else: l1.append(0)

            if w in second: l2.append(1)

            else: l2.append(0)
        # формула косинуса
        result = 1 - spatial.distance.cosine(l1, l2)
        res = {'first': row['text'], 'second': row1['text'], 'result': result}
        ls_res.append(res)
df_cos2 = pd.DataFrame(ls_res)
df_cos2.sort_values(by=['result'], ascending=False, inplace=True)
df_cos2.head(10)

'hey thanks for being top new followers this week! much appreciated happy'

In [ ]:
excel_lst = ['binar_token', 'binar_stemm', 'binar_lemm', 'count_token', 'count_stemm', 'count_lemm', 'tf_token', 'tf_stemm', 'tf_lemm']
df_lst = [df_token_binar, df_stemm_binar, df_lemm_binar, df_token, df_stemm, df_lemm, df_tf_tokenz, df_tf_stemm, df_tf_lemm]
excel_lst = ['binar_token', 'binar_stemm', 'binar_lemm']
df_lst = [df_token_binar, df_stemm_binar, df_lemm_binar]

In [ ]:
xl = pd.ExcelWriter('base2_tweets.xlsx', engine='xlsxwriter')
for i, defi in enumerate(df_lst):#прогоняем список чтобы загнать каждый датафрейм в отдельную страницу экселя
    defi.to_excel(
        xl,
        sheet_name=excel_lst[i],
        index=False)
xl.save()#сохранить в файл 293 517 083,45 

In [32]:
x = df_stemm_binar.drop(['0text0', '0status0'], axis=1)
y = df_stemm_binar['0status0']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

In [34]:
dtc = DecisionTreeClassifier(random_state=21, max_depth=20, criterion='gini')
dtc.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=20, random_state=21)

In [35]:
pred = dtc.predict(X_test)
accuracy_score(y_test, pred)

0.8933333333333333

In [37]:
x = df_lemm_binar.drop(['0text0', '0status0'], axis=1)
y = df_lemm_binar['0status0']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

In [39]:
svc = SVC(kernel='poly', probability=True, random_state=21, C=10)
svc.fit(X_train, y_train)

SVC(C=10, kernel='poly', probability=True, random_state=21)

In [41]:
pred = svc.predict(X_test)
accuracy_score(y_test, pred)

0.9383333333333334